## Data Summary:

The last column of the spam dataset denotes whether the e-mail was considered spam (1) or not (0), i.e. unsolicited commercial e-mail.
Most of the attributes indicate whether a particular word or character was frequently occuring in the e-mail.

The data is downloaded from UCI Machine Learning Repository. For more information, refer to: https://archive.ics.uci.edu/ml/datasets/spambase

## Introduction to Decision Tree:

A decision tree is a flowchart-like tree structure where an internal node represents feature(or attribute), the branch represents a decision rule, and each leaf node represents the outcome. The topmost node in a decision tree is known as the root node. It learns to partition on the basis of the attribute value. It partitions the tree in recursively manner call recursive partitioning. This flowchart-like structure helps you in decision making. It's visualization like a flowchart diagram which easily mimics the human level thinking. That is why decision trees are easy to understand and interpret.

### How does this work:
The basic idea behind any decision tree algorithm is as follows:

1. Select the best attribute using Attribute Selection Measures(ASM) to split the records.
2. Make that attribute a decision node and breaks the dataset into smaller subsets.
3. Starts tree building by repeating this process recursively for each child until one of the condition will match:
>* All the tuples belong to the same attribute value.
>* There are no more remaining attributes.
>* There are no more instances.

<img src="images/Fig1.png">

### Determine the best attribute test Condition
The decision tree inducing algorithm must provide a method for specifying the test condition for different attribute types as well as an objective measure for evaluating the good ness of each test condition.

First, the specification of an attribute test condition and its corresponding outcomes depends on the attribute types. We can do two-way split or multi-way split, discretize or group attribute values as needed. The binary attributes leads to two-way split test condition. For norminal attributes which have many values, the test condition can be expressed into multiway split on each distinct values, or two-way split by grouping the attribute values into two subsets. Similarly, the ordinal attributes can also produce binary or multiway splits as long as the grouping does not violate the order property of the attribute values. For continuous attributes, the test condition can be expressed as a comparsion test with two outcomes, or a range query. Or we can discretize the continous value into nominal attribute and then perform two-way or multi-way split.

Since there are may choices to specify the test conditions from the given training set, we need use a measurement to determine the best way to split the records. The goal of best test conditions is whether it leads a homogenous class distribution in the nodes, which is the purity of the child nodes before and after spliting. The larger the degree of purity, the better the class distribution.

To determine how well a test condition performs, we need to compare the degree of impurity of the parent before spliting with degree of the impurity of the child nodes after splitting. The larger their differnce, the better the test condition. The measurment of node impurity/purity are:
* Gini Index
* Entropy
* Misclassification Error

## Code:

In [109]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier #Decision Tree classifier
from sklearn.model_selection import KFold #for K fold crossvalidation
from sklearn.metrics import confusion_matrix #for confusion matrix
from sklearn.metrics import accuracy_score # for accuracy

Reading the data and renaming the last column as spam_class which denotes whether the e-mail was considered spam (1) or not (0), i.e. unsolicited commercial e-mail

In [110]:
data = pd.read_csv('data/spambase.data', header=None)
data.rename(columns={57:'spam_class'}, inplace=True)

In [111]:
y = np.array(data.pop('spam_class')) #y as output
X = np.array(data) #X as input

K Fold Cross Validation:
The general procedure is as follows:

1. Shuffle the dataset randomly.
2. Split the dataset into k groups
3. For each unique group:
>* Take the group as a hold out or test data set
>* Take the remaining groups as a training data set
>* Fit a model on the training set and evaluate it on the test set
>* Retain the evaluation score and discard the model
4. Summarize the skill of the model using the sample of model evaluation scores


In [112]:
kf = KFold(n_splits=10, random_state=None, shuffle=True)

### Entropy		
A decision tree is built top-down from a root node and involves partitioning the data into subsets that contain instances with similar values (homogenous). ID3 algorithm uses entropy to calculate the homogeneity of a sample. If the sample is completely homogeneous the entropy is zero and if the sample is an equally divided it has entropy of one.

<img src="images/Fig2.png">

### Information Gain		
The information gain is based on the decrease in entropy after a dataset is split on an attribute. Constructing a decision tree is all about finding attribute that returns the highest information gain (i.e., the most homogeneous branches).		
Step 1: Calculate entropy of the target. 		
Step 2: The dataset is then split on the different attributes. The entropy for each branch is calculated. Then it is added proportionally, to get total entropy for the split. The resulting entropy is subtracted from the entropy before the split. The result is the Information Gain, or decrease in entropy.
<img src="images/Fig3.png">
Step 3: Choose attribute with the largest information gain as the decision node, divide the dataset by its branches and repeat the same process on every branch.
Step 4a: A branch with entropy of 0 is a leaf node.
Step 4b: A branch with entropy more than 0 needs further splitting.		
Step 5: The ID3 algorithm is run recursively on the non-leaf branches, until all data is classified.

In [113]:
clf_entropy = DecisionTreeClassifier(criterion = "entropy")

In [114]:
df2 = pd.DataFrame()
misclassified = 0 
number_of_samples = 0
fpr = 0.0
fnr = 0.0
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index] #Split data
    y_train, y_test = y[train_index], y[test_index]
    clf_entropy = clf_entropy.fit(X_train,y_train)
    
    predictions = clf_entropy.predict(X_test) #calculating predictions
    
    accuracy = float(accuracy_score(y_test, predictions))
    error_rate = float(1 - accuracy)
    
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).flatten()
    fpr = float(fp)/float(tp + tn + fn + fp)
    fnr = float(fn)/float((tp + tn + fn + fp))
    
    number_of_samples += tp + tn + fn + fp
    misclassified += fp + fn
    
    df2 = df2.append(pd.DataFrame({"Error rate": [error_rate],"False positive rate": [fpr],"False negative rate ": [fnr]}),ignore_index=True)

In [115]:
print df2
print 'Overall error rate: ', float(misclassified)/float(number_of_samples)

   Error rate  False negative rate   False positive rate
0    0.060738              0.030369             0.030369
1    0.078261              0.043478             0.034783
2    0.115217              0.078261             0.036957
3    0.091304              0.026087             0.065217
4    0.082609              0.039130             0.043478
5    0.065217              0.034783             0.030435
6    0.078261              0.043478             0.034783
7    0.054348              0.023913             0.030435
8    0.065217              0.023913             0.041304
9    0.071739              0.034783             0.036957
Overall error rate:  0.0762877635297


#### Gini Index:
Gini index says, if we select two items from a population at random then they must be of same class and probability for this is 1 if population is pure.

1. It works with categorical target variable “Success” or “Failure”.
2. It performs only Binary splits
3. Higher the value of Gini higher the homogeneity.
4. CART (Classification and Regression Tree) uses Gini method to create binary splits.

Steps to Calculate Gini for a split
1. Calculate Gini for sub-nodes, using formula sum of square of probability for success and failure (p²+q²).
2. Calculate Gini for split using weighted Gini score of each node of that split

<img src="images/Fig4.png">

The Gini Index is calculated by subtracting the sum of the squared probabilities of each class from one.  It favors larger partitions.It considers a binary split for each attribute. You can compute a weighted sum of the impurity of each partition. If a binary split on attribute A partitions data D into D1 and D2, the Gini index of D is:
<img src="images/Fig5.png">

In [116]:
clf_gini = DecisionTreeClassifier(criterion = "gini")

In [117]:
df3 = pd.DataFrame()
misclassified = 0 
number_of_samples = 0
fpr = 0.0
fnr = 0.0
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index] #Split data
    y_train, y_test = y[train_index], y[test_index]
    clf_gini = clf_gini.fit(X_train,y_train)
    
    predictions = clf_gini.predict(X_test) #calculating predictions
    
    accuracy = float(accuracy_score(y_test, predictions))
    error_rate = float(1 - accuracy)
    
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).flatten()
    fpr = float(fp)/float(tp + tn + fn + fp)
    fnr = float(fn)/float((tp + tn + fn + fp))
    
    number_of_samples += tp + tn + fn + fp
    misclassified += fp + fn
    
    df3 = df3.append(pd.DataFrame({"Error rate": [error_rate],"False positive rate": [fpr],"False negative rate ": [fnr]}),ignore_index=True)

In [118]:
print df3
print 'Overall error rate: ', float(misclassified)/float(number_of_samples)

   Error rate  False negative rate   False positive rate
0    0.071584              0.039046             0.032538
1    0.082609              0.023913             0.058696
2    0.073913              0.043478             0.030435
3    0.089130              0.034783             0.054348
4    0.089130              0.032609             0.056522
5    0.076087              0.036957             0.039130
6    0.089130              0.045652             0.043478
7    0.100000              0.065217             0.034783
8    0.089130              0.030435             0.058696
9    0.086957              0.036957             0.050000
Overall error rate:  0.0847641816996
